In [1]:
from notebook_inits import *
from pathlib import Path
from main import _fetch_coinbase, _fetch_gemini, calculate_price
import pandas as pd

In [2]:
root == str(Path.cwd())

True

In [3]:
cb_data = _fetch_coinbase()

fetching coinbase data..


In [4]:
print(cb_data.keys())
print(cb_data['sequence'], cb_data['auction_mode'], cb_data['auction'], cb_data['time'])
('asks', len(cb_data['asks'])), ('bids', len(cb_data['bids']))

dict_keys(['bids', 'asks', 'sequence', 'auction_mode', 'auction', 'time'])
114357442688 False None 2025-10-28T11:33:16.270574570Z


(('asks', 18845), ('bids', 25938))

### Approach1, simple iterative: calculates execution price for every call

In [5]:
def calculate_price(orders:list, target_qty:float=10): # specific to coinbase
    # assuming orders are already sorted ascending by price
    remaining_qty: float = target_qty
    order_value: float = 0.0
    for order in orders:
        if remaining_qty <= 0:
            break
        order_price, order_qty = float(order[0]), float(order[1])
        available_qty  = min(order_qty, remaining_qty)
        order_value   += available_qty*order_price
        remaining_qty -= available_qty
        # print([order_price, order_qty, available_qty, remaining_qty])
    if remaining_qty > 0:
        print(f'\tPartial order:\n\tfilled={target_qty-remaining_qty};\n\t{remaining_qty=}')
       
    return order_value

In [6]:
%%timeit -r1 -n1
target_qt=100000
print(f'To buy  {target_qt} BTC= $', calculate_price(cb_data['asks'], target_qty=target_qt))
print('*'*12)
print(f'To sell {target_qt} BTC= $', calculate_price(cb_data['bids'], target_qty=target_qt))
print('='*25)

	Partial order:
	filled=4590.83144001021;
	remaining_qty=95409.16855998979
To buy  100000 BTC= $ 896370364.6837654
************
To sell 100000 BTC= $ 276815005.4342589
10.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Approach2, pandas.DataFrame: keeps the execution price pre-calculated & finds the price in between (minimal calculations)

In [7]:
# ASKS <-- lowest price: seller willing to accept; use it for buying
cols= ['price', 'qty', 'ods']
cb_asks = pd.DataFrame(cb_data['asks'], columns=cols)
del cb_asks['ods']
cb_asks[cols[:2]] = cb_asks[cols[:2]].astype(float)
cb_asks

,price,qty
0,1.145650e+05,2.409006e-02
1,1.145650e+05,1.852223e-02
2,1.145655e+05,2.618590e-02
3,1.145680e+05,1.710000e-02
4,1.145688e+05,6.705000e-05
...,...,...
18840,3.690808e+07,7.000000e-08
18841,9.999000e+07,1.632500e-04
18842,1.100239e+08,4.000000e-08
18843,1.238164e+08,1.100000e-07


In [8]:
# BIDS <-- highest price: buyers willing to pay; use it for selling
cols= ['price', 'qty', 'ods']
cb_bids = pd.DataFrame(cb_data['bids'], columns=cols)
del cb_bids['ods']
cb_bids[cols[:2]] = cb_bids[cols[:2]].astype(float)
cb_bids

,price,qty
0,114564.99,7.368043e-02
1,114564.14,1.293598e-02
2,114562.19,1.745770e-03
3,114562.18,2.619857e-02
4,114562.00,1.710000e-02
...,...,...
25933,0.05,1.519500e+04
25934,0.04,1.000000e+02
25935,0.03,3.442314e+04
25936,0.02,5.400000e+03


In [9]:
gem_data = _fetch_gemini()
gem_data.keys()

fetching gemini data..


dict_keys(['bids', 'asks'])

In [10]:
gem_bids, gem_asks = pd.DataFrame(gem_data['bids']), pd.DataFrame(gem_data['asks'])
del gem_bids['timestamp']
del gem_asks['timestamp']

In [11]:
gem_asks[:4]

,price,amount
0,114578.41,0.071046
1,114578.75,0.00093696
2,114580.0,3.0
3,114580.53,0.02291502


In [12]:
gem_bids[:4]

,price,amount
0,114558.77,0.02291502
1,114556.15,0.005
2,114554.45,0.00094191
3,114554.07,0.00436


In [13]:
gem_cols = ['price', 'qty']
gem_bids.rename(columns={'amount': 'qty'}, inplace=True)
gem_bids[gem_cols] = gem_bids[gem_cols].astype(float)

gem_asks.rename(columns={'amount': 'qty'}, inplace=True)
gem_asks[gem_cols] = gem_asks[gem_cols].astype(float)

In [14]:
gem_asks[:4]

,price,qty
0,114578.41,0.071046
1,114578.75,0.000937
2,114580.00,3.000000
3,114580.53,0.022915


In [15]:
gem_bids[:4]

,price,qty
0,114558.77,0.022915
1,114556.15,0.005000
2,114554.45,0.000942
3,114554.07,0.004360


In [16]:
merged_bids = pd.concat((cb_bids, gem_bids)).dropna().groupby('price').sum().reset_index()
merged_bids = merged_bids.sort_values(by='price', ascending=False, ignore_index=True)

In [17]:
merged_asks = pd.concat((cb_asks, gem_asks)).dropna().groupby('price').sum().reset_index()
merged_asks = merged_asks.sort_values(by='price', ascending=True, ignore_index=True)

In [18]:
merged_bids['cum_sum_qty'] = merged_bids['qty'].cumsum()
merged_bids['cum_sum_execution_price'] = (merged_bids['price']*merged_bids['qty']).cumsum()
# ---
merged_asks['cum_sum_qty'] = merged_asks['qty'].cumsum()
merged_asks['cum_sum_execution_price'] = (merged_asks['price']*merged_asks['qty']).cumsum()

In [19]:
merged_bids, len(merged_bids)

(           price           qty   cum_sum_qty  cum_sum_execution_price
 0      114564.99  7.368043e-02  7.368043e-02             8.441198e+03
 1      114564.14  1.293598e-02  8.661641e-02             9.923197e+03
 2      114562.19  1.745770e-03  8.836218e-02             1.012320e+04
 3      114562.18  2.619857e-02  1.145608e-01             1.312456e+04
 4      114562.00  1.710000e-02  1.316607e-01             1.508357e+04
 ...          ...           ...           ...                      ...
 25982       0.05  1.519500e+04  4.104088e+04             2.783720e+08
 25983       0.04  1.000000e+02  4.114088e+04             2.783720e+08
 25984       0.03  3.442314e+04  7.556402e+04             2.783730e+08
 25985       0.02  5.400000e+03  8.096402e+04             2.783731e+08
 25986       0.01  3.132694e+06  3.213658e+06             2.784045e+08
 
 [25987 rows x 4 columns],
 25987)

In [20]:
def calculate_execution_price_df(orders: pd.DataFrame, target_qt: float=10) -> float:
    if orders.empty:
        return 0
    insert_position = orders['cum_sum_qty'].searchsorted(target_qt)
    print(f">> {insert_position=}")
    print(orders[insert_position-1: insert_position+1])
    if insert_position == 0: # means lesser quantity than order book
        print(orders.iloc[0], 'target=', target_qt)
        return target_qt * orders.iloc[0]['price']

    if insert_position >= len(orders) - 1: # more than order book quantity, leads to partial fill the target_qty
        print(f"Partial fill --{target_qt=}; --remaining_qty={target_qt - orders.iloc[-1]['cum_sum_qty']}")
        return orders.iloc[-1]['cum_sum_execution_price']
    
    # somewhere in between the orders
    # print(orders[insert_position-1: insert_position+1])
    for idx, row in orders[insert_position-1: insert_position+1].iterrows():
        # find 2 exact rows where target_qty would fit
        print(f'processing row_idx={idx}')
        print('\n\t---', row['cum_sum_qty'])
        if row['cum_sum_qty'] == target_qt:
            print('------- found exact quantity')
            return row['cum_sum_execution_price']
        if row['cum_sum_qty'] < target_qt < orders.iloc[idx+1]['cum_sum_qty']:
            print(f'Found exact position: {(idx, idx+1)}')
            return row['cum_sum_execution_price'] + ((orders.iloc[idx+1]['cum_sum_qty'] - target_qt) * orders.iloc[idx+1]['price'])
    

In [23]:
%%timeit -n1 -r1
print(calculate_execution_price_df(merged_asks, 1))
print(calculate_execution_price_df(merged_bids, 1))

>> insert_position=np.int64(19)
        price       qty  cum_sum_qty  cum_sum_execution_price
18  114571.86  0.006121     0.985186             112871.49429
19  114572.00  0.017100     1.002286             114830.67549
processing row_idx=18

	--- 0.9851856999999998
Found exact position: (18, 19)
113133.37151084255
>> insert_position=np.int64(24)
        price       qty  cum_sum_qty  cum_sum_execution_price
23  114554.02  0.001694     0.929121            106441.190886
24  114554.01  0.290337     1.219459            139700.506600
processing row_idx=23

	--- 0.92912121
Found exact position: (23, 24)
131581.05698116223
5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [22]:
# calculate_execution_price_df(merged_bids, 0.1)